In [1]:
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import geopandas as gpd
from libpysal.weights import KNN
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm  # NEW: For gradient color mapping

In [2]:
# Load dataset (Manually set the file path for each category)
file_path = f"new_dataset\\Mammalia_top_species.csv"  # Change this for different categories
df = pd.read_csv(file_path)

# Ensure observed_date is in datetime format
df["observed_date"] = pd.to_datetime(df["observed_date"])

# Define dependent variable (species observation count)
df["species_count"] = df.groupby(["latitude", "longitude"])["scientific_name"].transform("count")
y = df["species_count"].values.reshape(-1, 1)  # Dependent variable

# Define independent variables (weather conditions)
X = df[["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max"]].values

# Define spatial coordinates (Latitude & Longitude)
coords = df[["longitude", "latitude"]].values

In [3]:
# Automatically find best bandwidth
bw = Sel_BW(coords, y, X, kernel="bisquare").search()
print(f"Optimal Bandwidth: {bw}")

Optimal Bandwidth: 97.0


In [4]:
# Run GWR with the selected bandwidth
gwr_model = GWR(coords, y, X, bw, kernel="bisquare")
gwr_results = gwr_model.fit()

# Print summary
print(gwr_results.summary())

Model type                                                         Gaussian
Number of observations:                                                5271
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          20240.055
Log-likelihood:                                                  -11025.141
AIC:                                                              22058.282
AICc:                                                             22060.294
BIC:                                                             -24898.006
R2:                                                                   0.000
Adj. R2:                                                             -0.000

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [5]:
# # Convert results into a DataFrame
# df["temp_coeff"] = gwr_results.params[:, 0]  # Coefficients for temperature

# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# # Plot the results as a heatmap
# fig, ax = plt.subplots(figsize=(10, 6))
# gdf.plot(column="temp_coeff", cmap="coolwarm", legend=True, ax=ax)
# plt.title("GWR Coefficients for Temperature Effect on Wildlife")
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()

In [6]:
# df["precip_coeff"] = gwr_results.params[:, 1]  # Coefficient for precipitation

# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))


# # Plot Precipitation Coefficients
# fig, ax = plt.subplots(figsize=(10, 6))
# gdf.plot(column="precip_coeff", cmap="coolwarm", legend=True, ax=ax)
# plt.title("GWR Coefficients for Precipitation Effect on Wildlife")
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()

In [7]:
# df["wind_coeff"] = gwr_results.params[:, 2]  # Coefficient for wind speed


# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# # Plot Wind Speed Coefficients
# fig, ax = plt.subplots(figsize=(10, 6))
# gdf.plot(column="wind_coeff", cmap="coolwarm", legend=True, ax=ax)
# plt.title("GWR Coefficients for Wind Speed Effect on Wildlife")
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()

In [8]:
# df["snow_coeff"] = gwr_results.params[:, 3]  # Coefficient for wind speed


# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# # Plot Wind Speed Coefficients
# fig, ax = plt.subplots(figsize=(10, 6))
# gdf.plot(column="snow_coeff", cmap="coolwarm", legend=True, ax=ax)
# plt.title("GWR Coefficients for Snow Effect on Wildlife")
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.show()

In [13]:
# Convert results into a DataFrame
df["temp_coeff"] = gwr_results.params[:, 0]  # Coefficients for temperature

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))



# ------------------ NEW: CREATE INTERACTIVE MAP WITH GRADIENT COLORS ------------------ #
# Initialize a folium map centered at the mean latitude and longitude
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# Create a color scale (blue for negative, red for positive, white near zero)
# colormap = cm.LinearColormap(
#     colors=["Black","Blue", "Aqua", "White"], 
#     vmin=df["temp_coeff"].min(), 
#     vmax=df["temp_coeff"].max(),
#     caption="Temperature Coefficient"
# )
# Create a diverging color map (red = positive, blue = negative)
colormap = cm.linear.RdBu_11.scale(gdf["temp_coeff"].min(), gdf["temp_coeff"].max())
colormap.caption = "Temperature Coefficient"

# Add points to the map with a gradient color scale
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color=colormap(row["temp_coeff"]),  # Assign color based on gradient
        fill=True,
        fill_color=colormap(row["temp_coeff"]),
        fill_opacity=0.7,
        popup=f"Temp Coeff: {row['temp_coeff']:.4f}",  # Show coefficient value on click
    ).add_to(m)

# Add the colormap legend to the map
m.add_child(colormap)

# Save the map as an HTML file and open it in the browser
m.save("Mammalia_temp_map_beforeCoords.html")
print("Map saved as gwr_temp_map.html - Open this file in your browser to view the interactive map.")

Map saved as gwr_temp_map.html - Open this file in your browser to view the interactive map.


In [10]:
# # Convert results into a DataFrame
# df["precip_coeff"] = gwr_results.params[:, 1]  # Coefficients for precipitation

# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# # ------------------ NEW: CREATE INTERACTIVE MAP WITH GRADIENT COLORS ------------------ #
# # Initialize a folium map centered at the mean latitude and longitude
# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# # Create a color scale (Black → Blue → Aqua → White)
# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["precip_coeff"].min(), 
#     vmax=df["precip_coeff"].max(),
#     caption="Precipitation Coefficient"
# )

# # Add points to the map
# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["precip_coeff"]),
#         fill=True,
#         fill_color=colormap(row["precip_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Precip Coeff: {row['precip_coeff']:.4f}",
#     ).add_to(m)

# # Add the colormap legend to the map
# m.add_child(colormap)

# # Save the map
# m.save("gwr_precip_map.html")
# print("Map saved as gwr_precip_map.html - Open this file in your browser to view the interactive map.")


In [11]:
# df["wind_coeff"] = gwr_results.params[:, 2]  # Coefficients for wind speed

# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["wind_coeff"].min(), 
#     vmax=df["wind_coeff"].max(),
#     caption="Wind Speed Coefficient"
# )

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["wind_coeff"]),
#         fill=True,
#         fill_color=colormap(row["wind_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Wind Coeff: {row['wind_coeff']:.4f}",
#     ).add_to(m)

# m.add_child(colormap)

# m.save("gwr_wind_map.html")
# print("Map saved as gwr_wind_map.html - Open this file in your browser to view the interactive map.")


In [12]:
# df["snow_coeff"] = gwr_results.params[:, 3]  # Coefficients for snowfall

# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["snow_coeff"].min(), 
#     vmax=df["snow_coeff"].max(),
#     caption="Snowfall Coefficient"
# )

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["snow_coeff"]),
#         fill=True,
#         fill_color=colormap(row["snow_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Snowfall Coeff: {row['snow_coeff']:.4f}",
#     ).add_to(m)

# m.add_child(colormap)

# m.save("gwr_snow_map.html")
# print("Map saved as gwr_snow_map.html - Open this file in your browser to view the interactive map.")
